In [15]:
# pip install earthengine-api geemap  # (run once in your environment)
import ee
ee.Initialize()  # run `earthengine authenticate` once in a terminal beforehand

# ========= USER INPUTS =========
lat, lon = 17.9246031, 73.7120122          # Example: New Delhi
buffer_m = 50                      # Radius around the point for stats
start, end = '2025-01-10', '2025-05-20'   # Period for NDVI & dynamic moisture

# ========= GEOMETRY =========
aoi = ee.Geometry.Point([lon, lat]).buffer(buffer_m).bounds()

# ========= SENTINEL-2 NDVI (context) =========
def s2_cloudmask(img):
    # Keep SCL classes except: 3 (cloud shadow), 8 (cloud med), 9 (cloud high), 10 (cirrus)
    scl = img.select('SCL')
    keep = scl.remap([3, 8, 9, 10], [0, 0, 0, 0], 1)
    return img.updateMask(keep)

s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(aoi).filterDate(start, end).map(s2_cloudmask))

ndvi = s2.median().normalizedDifference(['B8', 'B4']).rename('NDVI')

# ========= DYNAMIC SOIL MOISTURE (choose one) =========
# Option A: SMAP L4 (surface & root-zone cm3/cm3; 3-hourly -> averaged)
smap = (ee.ImageCollection('NASA/SMAP/SPL4SMGP/007')
        .filterDate(start, end).select(['sm_surface', 'sm_rootzone']))
sm_surface = smap.select('sm_surface').mean().rename('SM_surface')     # cm3/cm3
sm_rootzone = smap.select('sm_rootzone').mean().rename('SM_rootzone')  # cm3/cm3

# Option B: ERA5-Land daily (volumetric soil water by four layers) — uncomment if you prefer
# era5 = (ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
#         .filterDate(start, end)
#         .select(['volumetric_soil_water_layer_1',
#                  'volumetric_soil_water_layer_2',
#                  'volumetric_soil_water_layer_3',
#                  'volumetric_soil_water_layer_4']).mean()
# era5 = era5.rename(['SM_L1_0_7cm','SM_L2_7_28cm','SM_L3_28_100cm','SM_L4_100_289cm'])

# ========= STATIC SOIL PROPERTIES (OpenLandMap) =========
# Soil pH in water (x10); bands: b0, b10, b30, ... ; convert to true pH by *0.1
ph_img = ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').select(['b0','b10','b30']).multiply(0.1)
ph_top30 = ph_img.reduce(ee.Reducer.mean()).rename('pH_top30cm')

# Soil Organic Carbon (values are “×5 g/kg”); convert to g/kg by *5, to % by /10
soc_img = ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').select(['b0','b10','b30']).multiply(5.0)  # g/kg
soc_gkg_top30 = soc_img.reduce(ee.Reducer.mean()).rename('SOC_gkg_top30cm')
soc_pct_top30 = soc_gkg_top30.divide(10.0).rename('SOC_pct_top30cm')  # %

# Water content at 33 kPa (field capacity), volumetric %; a proxy for water-holding capacity
wc33_img = ee.Image('OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01').select(['b0','b10','b30'])
wc33_top30 = wc33_img.reduce(ee.Reducer.mean()).rename('WC33_vpct_top30cm')  # v%

# ========= ZONAL STATS =========
def mean_over(img, scale):
    return img.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=aoi, scale=scale, maxPixels=1e9)

summary = {}
summary.update({'NDVI_mean': mean_over(ndvi, 10).getInfo()['NDVI']})
summary.update(mean_over(sm_surface, 10000).getInfo())
summary.update(mean_over(sm_rootzone, 10000).getInfo())
summary.update(mean_over(ph_top30, 250).getInfo())
summary.update(mean_over(soc_gkg_top30, 250).getInfo())
summary.update(mean_over(soc_pct_top30, 250).getInfo())
summary.update(mean_over(wc33_top30, 250).getInfo())

print('Soil health summary over AOI:')
print(summary)
17.9246031, 73.7120122

# ========= OPTIONAL: STACKED IMAGE FOR EXPORT / MAPPING =========
soil_health = (ndvi
               .addBands(sm_surface)
               .addBands(sm_rootzone)
               .addBands(ph_top30)
               .addBands(soc_gkg_top30)
               .addBands(soc_pct_top30)
               .addBands(wc33_top30))

# Example: export GeoTIFF to Drive (uncomment in Colab / Notebook runtime)
# task = ee.batch.Export.image.toDrive(
#     image=soil_health,
#     description='soil_health_stack',
#     region=aoi,
#     scale=250,   # coarsest among static layers (change if exporting only NDVI)
#     maxPixels=1e13)
# task.start()


Soil health summary over AOI:
{'NDVI_mean': 0.46127950124962236, 'SM_surface': None, 'SM_rootzone': None, 'pH_top30cm': 5.6000000000000005, 'SOC_gkg_top30cm': 36.666666666666664, 'SOC_pct_top30cm': 3.6666666666666665, 'WC33_vpct_top30cm': 41.666666666666664}
